In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Creutzfeldt-Jakob_Disease"
cohort = "GSE87629"

# Input paths
in_trait_dir = "../../input/GEO/Creutzfeldt-Jakob_Disease"
in_cohort_dir = "../../input/GEO/Creutzfeldt-Jakob_Disease/GSE87629"

# Output paths
out_data_file = "../../output/preprocess/Creutzfeldt-Jakob_Disease/GSE87629.csv"
out_gene_data_file = "../../output/preprocess/Creutzfeldt-Jakob_Disease/gene_data/GSE87629.csv"
out_clinical_data_file = "../../output/preprocess/Creutzfeldt-Jakob_Disease/clinical_data/GSE87629.csv"
json_path = "../../output/preprocess/Creutzfeldt-Jakob_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on the Series_overall_design, this dataset contains DNA microarray analysis of B and T cells
is_gene_available = True  # DNA microarray data is gene expression data

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait: looking at row 5 which contains 'biopsy data, villus height to crypt depth'
# This measures the severity of the disease (villus atrophy) which can serve as our trait
trait_row = 5

# For age: There is no age information in the sample characteristics
age_row = None

# For gender: There is no gender information in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """
    Convert the villus height to crypt depth ratio to a continuous value.
    Higher values indicate healthier intestinal tissue (less disease severity).
    Lower values indicate more severe celiac disease activity.
    """
    if value is None:
        return None
    
    # Extract the numeric value after the colon
    if ':' in value:
        try:
            # The value is in format "biopsy data, villus height to crypt depth: X.X"
            return float(value.split(':')[1].strip())
        except (ValueError, IndexError):
            return None
    else:
        try:
            return float(value)
        except ValueError:
            return None

# Age and gender conversion functions are defined but won't be used
def convert_age(value):
    if value is None:
        return None
    
    if ':' in value:
        try:
            return float(value.split(':')[1].strip())
        except (ValueError, IndexError):
            return None
    else:
        try:
            return float(value)
        except ValueError:
            return None

def convert_gender(value):
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':')[1].strip().lower()
    else:
        value = value.lower()
    
    if value in ['female', 'f']:
        return 0
    elif value in ['male', 'm']:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save initial filtering information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Create a DataFrame from the Sample Characteristics Dictionary shown in the previous output
    sample_characteristics = {
        0: ['individual: celiac patient A', 'individual: celiac patient C', 'individual: celiac patient G', 'individual: celiac patient H', 'individual: celiac patient K', 'individual: celiac patient L', 'individual: celiac patient M', 'individual: celiac patient N', 'individual: celiac patient O', 'individual: celiac patient P', 'individual: celiac patient Q', 'individual: celiac patient R', 'individual: celiac patient S', 'individual: celiac patient T', 'individual: celiac patient U', 'individual: celiac patient V', 'individual: celiac patient W', 'individual: celiac patient X', 'individual: celiac patient Y', 'individual: celiac patient Z'],
        1: ['disease state: biopsy confirmed celiac disease on gluten-free diet greater than one year'],
        2: ['treatment: control', 'treatment: 6 weeks gluten challenge'],
        3: ['tissue: peripheral whole blood'],
        4: ['cell type: purified pool of B and T cells'],
        5: ['biopsy data, villus height to crypt depth: 2.9', 'biopsy data, villus height to crypt depth: 2.6', 'biopsy data, villus height to crypt depth: 1.1', 'biopsy data, villus height to crypt depth: 0.5', 'biopsy data, villus height to crypt depth: 0.3', 'biopsy data, villus height to crypt depth: 2', 'biopsy data, villus height to crypt depth: 0.4', 'biopsy data, villus height to crypt depth: 2.4', 'biopsy data, villus height to crypt depth: 1.4', 'biopsy data, villus height to crypt depth: 2.7', 'biopsy data, villus height to crypt depth: 3.5', 'biopsy data, villus height to crypt depth: 0.7', 'biopsy data, villus height to crypt depth: 0.2', 'biopsy data, villus height to crypt depth: 2.8', 'biopsy data, villus height to crypt depth: 3', 'biopsy data, villus height to crypt depth: 0.8', 'biopsy data, villus height to crypt depth: 1.2', 'biopsy data, villus height to crypt depth: 1.7', 'biopsy data, villus height to crypt depth: 2.5', 'biopsy data, villus height to crypt depth: 2.1', 'biopsy data, villus height to crypt depth: 3.1'],
        6: ['hybridization batch: 1']
    }
    
    # Convert the dictionary to a DataFrame
    clinical_data = pd.DataFrame.from_dict(sample_characteristics, orient='index')
    
    # Extract clinical features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of clinical data:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


### Step 4: Gene Identifier Review

In [ ]:
# From the identifiers shown, we can observe that the gene identifiers are in the format "ILMN_xxxxxxx".
# This format indicates that they are Illumina probe IDs, not standard human gene symbols.
# Illumina probe IDs need to be mapped to human gene symbols for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify the appropriate column names for gene identifiers and gene symbols
# From the gene_annotation preview, we see:
# - 'ID' column contains Illumina probe IDs (e.g., ILMN_1725881) which match our gene expression data index
# - 'Symbol' column contains the gene symbols we want to map to (e.g., LOC23117, FCGR2B, TRIM44)

# 2. Get gene mapping dataframe by extracting the two columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# Print the first few rows to verify the mapping
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# Using the library function that distributes probe values across multiple genes and sums by gene
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the first few rows to verify the mapped gene expression data
print("\nGene expression data preview (after mapping):")
print(preview_df(gene_data))

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data
# First, let's read the clinical data that was saved in step 2
clinical_df = pd.read_csv(out_clinical_data_file)

# The clinical data is loaded but it might not have the trait name as a column
# We need to set the first column as the trait column
clinical_df.columns = [trait]
print("Clinical data preview:")
print(clinical_df.head())

# Create a transposed version for linking with gene data (samples as columns)
clinical_df_t = clinical_df.T
print("Transposed clinical data shape:", clinical_df_t.shape)

# 3. Link the clinical and genetic data
linked_data = pd.concat([clinical_df_t, normalized_gene_data]).T
print(f"Linked data shape (before handling missing values): {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Data after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct final quality validation and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains villus height to crypt depth ratio measurements in celiac disease patients, which reflects disease severity."
)

# 7. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical features using the functions defined in step 2
# First, let's load the clinical data again to ensure we have the latest version
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Extract clinical features using the correct trait name from the variable
selected_clinical_df = geo_select_clinical_features(
    clinical_data, 
    trait=trait, 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape (before handling missing values): {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Data after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are biased
# Check if trait is biased
trait_type = 'binary' if len(linked_data[trait].unique()) == 2 else 'continuous'
if trait_type == "binary":
    is_trait_biased = judge_binary_variable_biased(linked_data, trait)
else:
    is_trait_biased = judge_continuous_variable_biased(linked_data, trait)

# Remove biased demographic features if present
unbiased_linked_data = linked_data.copy()
if "Age" in unbiased_linked_data.columns:
    age_biased = judge_continuous_variable_biased(unbiased_linked_data, 'Age')
    if age_biased:
        print(f"The distribution of the feature 'Age' in this dataset is severely biased.")
        unbiased_linked_data = unbiased_linked_data.drop(columns='Age')
    else:
        print(f"The distribution of the feature 'Age' in this dataset is fine.")

if "Gender" in unbiased_linked_data.columns:
    gender_biased = judge_binary_variable_biased(unbiased_linked_data, 'Gender')
    if gender_biased:
        print(f"The distribution of the feature 'Gender' in this dataset is severely biased.")
        unbiased_linked_data = unbiased_linked_data.drop(columns='Gender')
    else:
        print(f"The distribution of the feature 'Gender' in this dataset is fine.")

# 6. Conduct final quality validation and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains villus height to crypt depth ratio measurements in celiac disease patients, which reflects disease severity when studied for Creutzfeldt-Jakob_Disease."
)

# 7. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")